<a href="https://colab.research.google.com/github/mukulsn/Machine-Learning/blob/main/Projects/CI/scripts/scraping_try.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

good source to get end of the webpage of infinte scroll
https://www.benjaminrancourt.ca/how-to-get-to-the-end-of-a-page-with-infinite-scrolling/#:~:text=scrollHeight%3B%20%2F%2F%20Scroll%20at%20the,load%20if%20(currentScrollHeight%20%3D%3D%3D%20document.

In [1]:
# 1px = 0.026458 cm
1/0.026458

37.79575175750246

# setup for scraping with selenium

In [1]:
!wget https://dl.google.com/linux/direct/google-chrome-stable_current_amd64.deb

--2024-03-16 11:45:56--  https://dl.google.com/linux/direct/google-chrome-stable_current_amd64.deb
Resolving dl.google.com (dl.google.com)... 173.194.74.136, 173.194.74.91, 173.194.74.190, ...
Connecting to dl.google.com (dl.google.com)|173.194.74.136|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 106053828 (101M) [application/x-debian-package]
Saving to: ‘google-chrome-stable_current_amd64.deb’

google-chrome-stabl 100%[===================>] 101.14M   429MB/s    in 0.2s    

2024-03-16 11:45:57 (429 MB/s) - ‘google-chrome-stable_current_amd64.deb’ saved [106053828/106053828]



In [2]:
!sudo apt install /content/google-chrome-stable_current_amd64.deb

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
Note, selecting 'google-chrome-stable' instead of '/content/google-chrome-stable_current_amd64.deb'
The following additional packages will be installed:
  libu2f-udev libudev1 libvulkan1 mesa-vulkan-drivers systemd-hwe-hwdb udev
The following NEW packages will be installed:
  google-chrome-stable libu2f-udev libvulkan1 mesa-vulkan-drivers
  systemd-hwe-hwdb udev
The following packages will be upgraded:
  libudev1
1 upgraded, 6 newly installed, 0 to remove and 37 not upgraded.
Need to get 12.5 MB/119 MB of archives.
After this operation, 401 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 libudev1 amd64 249.11-0ubuntu3.12 [78.2 kB]
Get:2 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 udev amd64 249.11-0ubuntu3.12 [1,557 kB]
Get:3 http://archive.ubuntu.com/ubuntu jammy/main amd64 libu2f-udev all 1.1.10-3build2 [4,190 B]
Get:

In [3]:
!pip install selenium

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.0/10.0 MB 32.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 460.2/460.2 kB 53.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 9.7 MB/s eta 0:00:00


In [4]:
!pip install webdriver-manager

# Main code

In [ ]:
import urllib.request
import numpy as np
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import ElementNotInteractableException
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager
import time
import pandas as pd
import re
import os
from datetime import datetime

# datetime object containing current date and time
start = datetime.now()

#options = webdriver.ChromeOptions()
#options.add_experimental_option("detach", True)
#options.add_argument("--start-maximized")
#driver=webdriver.Chrome(options=options, service=Service(ChromeDriverManager().install()))

chrome_options = Options()
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--headless=new')
chrome_options.add_argument('--disable-dev-shm-usage')
chrome_options.add_argument('--start-maximized')
user_agent = 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/122.0.0.0 Safari/537.36'
chrome_options.add_argument(f'user-agent={user_agent}')
# # ######################
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()),options=chrome_options)


def driver_click(xpath):
    element = driver.find_element(By.XPATH, xpath)
    # driver.execute_script('arguments[0].scrollIntoView();')
    driver.execute_script('window.scrollBy(0,-200);')
    element.click()


def absolute_number(num):
    a = str(num)
    if a.find('k') == -1:
        return a
    elif a.find('k') != 1:
        return int(float(a[:-1]) * 1000)

def extract_pid(text,pattern):
    return text.rfind(pattern,0,text.rfind(pattern))

def page_number_extract(page):
    var = None
    page_number = page.find('=')
    if page_number == -1:
        var = 1
    else:
        var = page[page_number+1:]
    return var

def working_data(soup):
    # bigbox = soup.findAll("li",{"class":"product-base"})
    prod_meta_info = soup.findAll("div",{"class":"product-productMetaInfo"})
    prod_rating = soup.findAll("div",{"class":"product-ratingsContainerproduct-ratingsContainer"})
    prod_image = soup.findAll("div",{"class":"product-imageSliderContainer"})
    pagination_number =soup.findAll("ul",{"class":"pagination-container"})
    #new_season = soup.findAll('div',{'class':'xcelerator-plpxceleratorImageTag'})

    all_sku = soup.select('div[class="contentHolder"]')
    all_image = soup.select('div[class= "imgHolder"]')

    pid_url =soup.select('div[class="item rilrtl-products-list__item item"]')


# new_season = soup.findAll('div',{'class':'results-showMoreContainer'})

    href = []
    brand = []
    product = []
    prod_dp =[]
    prod_sp = []
    prod_discp = []
    prod_rating = []
    prod_ratingc = []
    prod_img = []
    page_number = []
    season_tag = []
    few_left = []
    total_items = []
    pid_list = []
    prod_ratingc_abs = []
    pp_price = []
    offer_price=[]
    exclusive=[]

    for i in range(len(all_sku)): # add iferror in the code
        print(i)
        # link= "https://www.myntra.com/"+bigbox[i].a['href']
        # pid = link[extract_pid(link,'/'):]
        # pid = pid[1:pid.rfind('/')]
        image_holder = all_image[i]
        link = "https://www.ajio.com" + pid_url[i].a['href']
        try:
            pid = pid_url[i].a['href'].rsplit('/',1)[1]
        except:
            pid = None
        title_brand= all_sku[i].find('div',{'class': 'brand'}).text #
        title_product= all_sku[i].find('div', {'class':'nameCls'}).text #
        try:
            pp = prod_meta_info[i].find("div", {'class': 'product-price'}).text
        except:
            pp = None
        try:
            dp = all_sku[i].find('span', {'class':"price"}).text
            dp = dp.replace(chr(8377),"").replace(',','')
        except:
            dp = None
        try:
            sp= all_sku[i].find('span', {'class':"orginal-price"}).text
            sp = sp.replace(chr(8377), "").replace(',','')
        except:
            sp = None
        try:
            of_p = all_sku[i].find('span',{'class': "offer-pricess"}).text
            of_p = of_p.replace(chr(8377),"").replace(',','').replace(' off','')
        except:
            of_p = None
        try:
            product_discount_percentage =all_sku[i].find('span', {'class':"discount"}).text
            product_discount_percentage = product_discount_percentage.replace('(','').replace(')','').replace(' off','')
        except:
            product_discount_percentage = None
        try:
            excl =all_image[i].find('div', {'class':'exclusive'}).text
        except:
            excl = None
        try:
            product_rating= all_sku[i].findAll("div", {"class":"product-ratingsContainer"})[0].span.text
        except:
            product_rating= None
        try:
            product_rating_count= all_sku[i].findAll("div", {"class":"product-ratingsCount"})[0].text[1:] #product_rating_count absolute_number(product_rating_count)
        except:
            product_rating_count =None
        try:
            product_rating_count_abs =absolute_number(product_rating_count)
        except:
            product_rating_count_abs = None
        try:
            product_image= image_holder.find('img',{'class':"rilrtl-lazy-img rilrtl-lazy-img-loaded"})["src"]
        except:
            product_image = None
        try:
            pg_number = pagination_number[0].findAll('11', {'class': 'pagination-active'})[0].a["href"]
            pg_number=page_number_extract(pg_number)
        except:
            pg_number = None
        try:
            season_tag_var =prod_image[i].div.find("div", {"class":"xcelerator-plpXceleratorImageTag"}).text
        except:
            season_tag_var= None
        try:
            few_left_var= prod_meta_info[i].find('div', {'class':'xcelerator-plpXceleratorInfoTag'}).text
        except:
            few_left_var=None
        try:
            total_itms = soup.find('div',{'class':'filter-container'}).div.div.text
        except:
            total_itms=None



        href.append(link)
        brand.append(title_brand)
        product.append(title_product)
        prod_dp.append(dp)
        prod_sp.append(sp)
        prod_discp.append(product_discount_percentage)
        prod_rating.append(product_rating)
        prod_ratingc.append(product_rating_count)
        prod_img.append(product_image)
        page_number.append(pg_number)
        season_tag.append(season_tag_var)
        few_left.append(few_left_var)
        total_items.append(total_itms)
        prod_ratingc_abs.append(product_rating_count_abs)
        pid_list.append(pid)
        pp_price.append(pp)
        offer_price.append(of_p)
        exclusive.append(excl)

    df =pd.DataFrame({'Brand':brand,
                        'Link':href,
                        'PID':pid_list,
                        'Product':product,
                        # 'product_price':pp_price,
                        'Dp':prod_dp,
                        'Sp':prod_sp,
                        'offer_price_AJIO':offer_price,
                        'Exclusive_AJIO':exclusive,
                        'Product_discount_percentage':prod_discp,
                        # 'Product_rating':prod_rating,
                        # 'prod_rating_count':prod_ratingc,
                        # 'prod_rating_count_abs':prod_ratingc_abs,
                        'product_image_url':prod_img,
                        # 'page number':page_number,
                        # 'season_tag':season_tag,
                        # 'only_few_left':few_left,
                        'total_items':total_items})

    # #adding product rating count absolute numbers# df['product_rating_count_abs'] = absolute_number(product_rating_count)PTD# df['PID'] = pid[1:pid.rfind('/')]

    return df


# Define a function to check if all images are loaded
def are_all_images_loaded():
    try:
        # Find all image elements on the page
        img_elements = driver.find_elements(By.TAG_NAME, "img")

        # Check if all images are loaded
        for img in img_elements:
            if not img.get_attribute("complete"):
                return False
        return True
    except:
        return False

def scroll_till_end(driver):

    # Scroll the page gradually
    scroll_speed = 120 # Adjust scroll speed as needed (milliseconds)
    scroll_pause_time = 0.1 # Adjust pause time as needed (seconds)
    current_scroll_position = 0

    page_height = driver.execute_script("return document.body.scrollHeight")
    print(f'page height {page_height}')

    while current_scroll_position < page_height:
      # time.sleep(1.2)
      # print(f'page height {page_height}')
      # for i in range(3):
      driver.execute_script("window.scrollTo(0, {});".format(current_scroll_position))
      current_scroll_position += scroll_speed
      time.sleep(scroll_pause_time)
      page_height = driver.execute_script("return document.body.scrollHeight")
      print(f'current scroll position {current_scroll_position}')
        # if page_height > 10000:
        #     scroll_speed = 40 # Adjust scroll speed as needed (milliseconds)
        #     scroll_pause_time = 0.1 # Adjust pause time as needed (seconds)


        # ####### EXPERIMENTING WITH PAGE WAIT STRATEGY
        # # Check if all images are loaded
        # if are_all_images_loaded():
        #     print("Some images failed to load.")
        #     time.sleep(3)

        #     # now reverse scrolling for 3 times
        #     for i in range(3):
        #         # time.sleep(1)
        #         current_scroll_position -= scroll_speed
        #         driver.execute_script("window.scrollTo(0, {});".format(current_scroll_position))
        #         # time.sleep(scroll_pause_time)
        #     time.sleep(scroll_pause_time)
        # else:
        #     print("All images are loaded properly.")
# %%

main_data = pd.DataFrame()
for i in range(1): # because 20 dresses have 2663 on myntra
    driver.get('https://www.ajio.com/c/830303011?query=%3Arelevance%3Abrand%3ASHOWOFF&gridColumns=5&segmentIds=')  # dress material
    # driver.get('https://www.ajio.com/c/830303011')  # kurta sets women
    time.sleep(2)
    driver_click('//*[@id="products"]/div[3]/div/div[2]/div/div[2]')
    # if i == 0:
    #     driver.get('https://www.myntra.com/20dresses')
    # else:
    #     driver.get(f'https://www.myntra.com/20dresses&p={i+1}')
    print('new script')
    print('page loaded')
    scroll_till_end(driver)
    print('scroll completed')
    soup = BeautifulSoup(driver.page_source, features="html.parser")
    df = working_data(soup)
    main_data=pd.concat([main_data,df])
    print(main_data)
    main_data.to_csv(r'data showoff ajio.csv')
    break
driver.close()
# main_data.to_csv(r'data\ajio data experiment.csv')

# %%
end = datetime.now()

print(f'start time {start} and end time {end}')

#BECAUSE IMAGE OF AJIO WAS NOT AVAILABLE IN DATA SO THIS CODE IS TO GET IMAGES FROM PDP PAGE OF AJIO

In [26]:
import urllib.request
import numpy as np
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import ElementNotInteractableException
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager
import time
import pandas as pd
import re
import os
from datetime import datetime

# datetime object containing current date and time
start = datetime.now()

#options = webdriver.ChromeOptions()
#options.add_experimental_option("detach", True)
#options.add_argument("--start-maximized")
#driver=webdriver.Chrome(options=options, service=Service(ChromeDriverManager().install()))

chrome_options = Options()
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--headless=new')
chrome_options.add_argument('--disable-dev-shm-usage')
chrome_options.add_argument('--start-maximized')
user_agent = 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/122.0.0.0 Safari/537.36'
chrome_options.add_argument(f'user-agent={user_agent}')
# # ######################
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()),options=chrome_options)


In [20]:
def extract_image(soup):
    abc = []
    for i in soup.findAll('div',{'class':'img-container'}):
        abc.append(str(i.findChildren()))

    for i in abc:
        if 'class="rilrtl-lazy-img' in i:
            # pos = i.find('src')
            pos = i.find('src')
            i[pos+5:].find('"')
            print(i[pos+5:pos+5+i[pos+5:].find('"')])
            return(i[pos+5:pos+5+i[pos+5:].find('"')])


# a = 'https://www.ajio.com/showoff-floral-print-anarkali-kurta-with-mandarin-collar/p/464001959_white'
# driver.get(a)
# b = BeautifulSoup(driver.page_source, features="html.parser")
# extract_image(b)

In [32]:
df = pd.read_csv('/content/data showoff ajio.csv')
df = df.drop(columns=['Unnamed: 0'])
df =df.drop_duplicates()
link_list = [(pid,link) for pid,link in zip(df['PID'],df['Link'])]


In [33]:
start = datetime.now()

new_list = []
count = 0
for pid,link in link_list:

  a = link
  try:
    driver.get(a)
  except:
    driver.close()
    driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()),options=chrome_options)
    # time.sleep(2)
    driver.get(a)
  b = BeautifulSoup(driver.page_source, features="html.parser")
  final_link = extract_image(b)
  result = (pid,link,final_link)
  count+=1
  if count%5==0:
    time.sleep(2)
    print(f'sleeping for 2 seconds, count{count}')
  new_list.append(result)


end = datetime.now()

print(f'start time {start} and end time {end}')

https://assets.ajio.com/medias/sys_master/root/20220314/AaCG/622f81eef997dd03e211a50a/-473Wx593H-464001959-white-MODEL.jpg
https://assets.ajio.com/medias/sys_master/root/20230828/HC5T/64ec7661ddf779151980ebc5/-473Wx593H-466501065-yellow-MODEL.jpg
https://assets.ajio.com/medias/sys_master/root/20230826/i6Qs/64e95355ddf77915197ed384/-473Wx593H-466500114-rust-MODEL.jpg
https://assets.ajio.com/medias/sys_master/root/20230828/9CZ6/64ecca69afa4cf41f5914363/-473Wx593H-466501540-cream-MODEL.jpg
https://assets.ajio.com/medias/sys_master/root/20230621/5yih/6492a55e42f9e729d76bdfdb/-473Wx593H-463949944-yellow-MODEL.jpg
sleeping for 2 seconds, count5
https://assets.ajio.com/medias/sys_master/root/20230624/aaKx/6496b990a9b42d15c9e7162d/-473Wx593H-465343328-yellow-MODEL.jpg
https://assets.ajio.com/medias/sys_master/root/20230828/hgdx/64ec7aecafa4cf41f58f3b16/-473Wx593H-466503708-cream-MODEL.jpg
https://assets.ajio.com/medias/sys_master/root/20230828/UDB4/64eccc35ddf779151984071f/-473Wx593H-466509880

In [35]:
fetched_data = pd.DataFrame()
fetched_data['pid']=[x[0] for x in new_list]
fetched_data['link']=[x[1] for x in new_list]
fetched_data['product_image_url']=[x[2] for x in new_list]


In [ ]:
urls = {x:y for x,y in zip(fetched_data['pid'],fetched_data['product_image_url'])}
urls

In [42]:
df['product_image_url'] = df['PID'].map(urls)

In [43]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1501 entries, 0 to 1500
Data columns (total 11 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   Brand                        1501 non-null   object 
 1   Link                         1501 non-null   object 
 2   PID                          1500 non-null   object 
 3   Product                      1501 non-null   object 
 4   Dp                           1501 non-null   int64  
 5   Sp                           1501 non-null   int64  
 6   offer_price_AJIO             1501 non-null   int64  
 7   Exclusive_AJIO               0 non-null      float64
 8   Product_discount_percentage  1501 non-null   object 
 9   product_image_url            1500 non-null   object 
 10  total_items                  1501 non-null   object 
dtypes: float64(1), int64(3), object(7)
memory usage: 140.7+ KB


In [44]:
df.to_csv('ajio data.csv')